In [ ]:
# Sample code for 07-predictive-analysis.ipynb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv('../data/processed_data/falcon9_processed.csv')

# Prepare features and target
X = df[['payload_mass_kg', 'orbit', 'launch_site', 'flight_number', 'reused']]
y = df['landing_success']

# Define preprocessing for numerical and categorical features
numerical_features = ['payload_mass_kg', 'flight_number']
categorical_features = ['orbit', 'launch_site', 'reused']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model pipelines
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

# Train models
rf_pipeline.fit(X_train, y_train)
lr_pipeline.fit(X_train, y_train)

# Make predictions
rf_pred = rf_pipeline.predict(X_test)
lr_pred = lr_pipeline.predict(X_test)

# Evaluate models
print("Random Forest Results:")
print(classification_report(y_test, rf_pred))

print("\nLogistic Regression Results:")
print(classification_report(y_test, lr_pred))

# Create confusion matrix
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_test, rf_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Random Forest Confusion Matrix')

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, lr_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Logistic Regression Confusion Matrix')

plt.tight_layout()
plt.savefig('../images/visualizations/confusion_matrices.png')
plt.show()